In [61]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from collections import defaultdict, Counter

import pandas as pd

In [3]:
data_dir = "./data/"

In [10]:
cans = pd.read_csv(data_dir + 'cans.csv')
cans

,vid,vid_
0,5bc78a50602b520bb3f6c87e3c542f1c,53634
1,73e37445d73561ffdf0711a3ffe4ca25,67745
2,bb639a67990c3b4ab3b51f7f7b1dbbb0,109429
3,29d0413f14eb1fbd8b5b43946b77f73a,24172
4,773e052e4aa8f09ed9f54f1f2699fc07,69757
...,...,...
13401,f3c19753a76e0e97f92dc715a8ee8c2b,142468
13402,607619c7529cf88720598c96402c0703,56365
13403,8e8b692d0670cca0099483ac0310f62e,83309
13404,f865e9c6257a4f6e811a9b4dc2b4ed75,145178


In [14]:
with open(data_dir + 'history.pkl', 'rb') as f:
    did_his = pickle.load(f)

# Overview
基于规则的方法效果有限，还是得上模型啊。既然要训模型，那么肯定是要构造数据集的。细想一下，可能主要要解决的问题是：
- 构造用于训模型的数据集；
- 为每个用户召回一定量的物品；

# 召回
召回主要是基于用户信息、物品信息、用户行为信息，在当前环境下，召回一组物品。一些召回方法：
- 关联的召回；
- 热门召回；
- 类别 / 风格召回；
- 图上的随机游走召回；
- 向量召回；

召回的结果可以用于：
- 构造训练集；
- 也可以用于为每个用户召回一定量的结果然后进行打分排序后作为提交结果；  
第二种怎么做是很明了的，但是怎么构造训练集呢？

## 关联召回

In [7]:
with open(data_dir + 'edges_.pkl', 'rb') as f:
    edges_ = pickle.load(f)
    out_edges_ = edges_['out']
    in_edges_  = edges_['in']

In [21]:
# 找到每个用户最后一次观看的视频
rec_link = defaultdict(list)
cans_vid = set(cans['vid_'])
null_did_link = []

for i, (k, v) in enumerate(did_his.items()):
    tmp = []
    v = v[-1]
    if v in out_edges_:
        # tmp = [e[0] for e in out_edges_[v].most_common()]
        for e in out_edges_[v].most_common():
            if e[0] not in tmp and e[0] in cans_vid:
                tmp.append(e[0])
        
    if len(tmp) < 6 and v in in_edges_:  # 如果找不到 6 个，则用其入边填充
        # n = 6 - len(tmp)
        for e in in_edges_[v].most_common():
            if e[0] not in tmp and e[0] in cans_vid:
                tmp.append(e[0])
    
    
    rec_link[k] = tmp
    if len(tmp) == 0:
        # print('Link recall get nothing for user: %d' % k)
        null_did_link.append(k)
    
    if i % 10000 == 0:
        print('%d / %d' % (i, len(did_his)))

print('Link recall get nothing for %d dids: %s' % (len(null_did_link), null_did_link))

0 / 170909
10000 / 170909
20000 / 170909
30000 / 170909
40000 / 170909
50000 / 170909
60000 / 170909
70000 / 170909
80000 / 170909
90000 / 170909
100000 / 170909
110000 / 170909
120000 / 170909
130000 / 170909
140000 / 170909
150000 / 170909
160000 / 170909
170000 / 170909
Link recall get nothing for 971 dids: [57, 327, 331, 543, 765, 871, 1023, 1073, 1529, 1595, 1612, 1685, 1839, 1879, 2036, 2196, 2393, 2566, 2691, 2916, 3078, 3268, 3286, 3504, 3595, 3684, 3777, 3986, 4005, 4143, 4508, 4552, 4571, 4698, 5061, 5302, 5886, 5902, 6201, 6268, 6907, 6945, 7073, 7133, 7181, 7580, 7630, 7650, 8314, 8389, 8418, 8775, 9441, 9831, 9847, 9910, 10248, 10326, 10616, 10683, 10702, 10813, 11028, 11104, 11109, 11729, 11766, 11785, 11943, 12043, 12178, 12204, 12536, 12649, 13327, 13341, 13620, 13735, 14563, 14639, 15108, 15628, 15789, 15994, 16073, 16311, 16331, 16401, 16476, 16606, 16767, 16846, 16900, 17425, 17610, 17876, 18031, 18196, 18541, 18634, 18715, 19181, 19352, 19409, 19729, 19758, 19909, 1

## 热门召回

In [26]:
with open(data_dir + 'hot_list.pkl', 'rb') as f:
    hot_list = pickle.load(f)

In [28]:
rec_hot = {}
cans_vid = set(cans['vid_'])
null_did_hot = []

# for i, (k, v) in enumerate(did_his.items()):
    

## 类别/风格召回
类别/风格可以是合集、视频类型、系列、频道类型、视频的明星集合、关键词、标签 (好吧，基本上把视频的属性全都包括进行来了），这有点类似于搜索中的基于 term 的匹配了，一种硬匹配。

## 随机游走召回
构建 (有向) 图，在图上进行随机游走。怎么构建这个图呢？
- 一种简单的做法是基于关联数据构建；
- **?**

## 向量召回
获得视频的表征向量，通过向量的 KNN 来做召回。虽然获得视频向量时，利用的信息大部分还是来自视频的信息，但与类被/风格召回不同之处在于：向量召回能够发掘视频啊之间的深层关系/语义关系，是一种软匹配。

## 随机召回


## 衡量召回的效果
怎么知道我召回的效果好坏呢？

# 构造训练集
这里的训练集指的是用于训练打分模型的数据集。其样本构造为 $(did, context, vid, label)$，其中 $label \in \{0, 1\}$，表示是否观看。需要斟酌一下 context 该如何定义：
- 用户的观看行为序列；
- 相关的统计特征；  
这 context 还挺难搞的啊，算了，先把数据切分成一组一组的吧。**由于按照用户进行分组后，每个用户的行为数据都变成了一个序列，例如观看过的视频是一个序列、vts 也是一个序列等，如何将这些序列变成一个特征，或者变成一个向量**：
- 基于这些序列构建对应的统计特征，例如行为序列中观看的视频的类型的分布；
- Embedding 后进行聚合，例如 Mean, Max, Concat 等；

正样本集，一种比较直接的方式是根据行为序列来构建，即序列中的每个物品和用户构成一个正样本，重点是如何构造负例。构造负例的方法：
- 随机选取热门物品作为负例；
- 召回后的负例；

In [25]:
main = pd.read_csv(data_dir + 'main.csv').iloc[:, 2:]
main

,vts,hb,seq_no,cpn,fpn,time_gap,did_,vid_
0,111.0,84.0,5.0,1,130,87.0,63411,77036
1,114.0,85.0,4.0,1,130,115.0,63411,14321
2,137.0,80.0,3.0,1,130,75.0,63411,23052
3,91.0,74.0,2.0,1,130,90.0,63411,24779
4,118.0,86.0,1.0,1,130,NaN,63411,86561
...,...,...,...,...,...,...,...,...
5596408,27.0,2456.0,5.0,1,130,33.0,121675,78186
5596409,52.0,2351.0,4.0,1,130,42.0,121675,139535
5596410,51.0,2064.0,3.0,1,130,5.0,121675,143599
5596411,186.0,2033.0,2.0,1,68,160376.0,121675,95951


In [35]:
g = main.groupby('did_')

In [59]:
def generate_negtive(his_df):
    """
    根据一个用户的历史行为生成负样本
    """
    
    pass


def generate_positive(his_df):
    """
    根据一个用户的历史行为生成正样本
    """
    his_df.sort_values('seq_no')
    context = {}
    last = his_df.iloc[-1]['vid_']
    his_df = his_df.iloc[:-1]
    cols = ['vts', 'hb', 'seq_no', 'cpn', 'fpn', 'time_gap']
    for col in cols:
        context[col] = his_df[col].values
    sample = {}
    sample['did'] = his_df.iloc[0]['did_']
    sample['vid'] = his_df.iloc[0]['vid_']
    sample['context'] = context
    sample['label'] = 1
    
    return sample

In [56]:
generate_positive(g.get_group(5))

{'did': 5.0,
 'vid': 7358.0,
 'context': {'vts': array([3960., 1702.,  779., 2468.]),
  'hb': array([6214., 1823., 2023., 2917.]),
  'seq_no': array([5., 4., 3., 2.]),
  'cpn': array([1, 1, 1, 1]),
  'fpn': array([ 68,  68, 130, 130]),
  'time_gap': array([3.81938e+05, 9.09000e+02, 6.43181e+05, 2.00000e+00])},
 'label': 1}

In [60]:
g.get_group(5)

,vts,hb,seq_no,cpn,fpn,time_gap,did_,vid_
2189596,3960.0,6214.0,5.0,1,68,381938.0,5,7358
2189597,1702.0,1823.0,4.0,1,68,909.0,5,124574
2189598,779.0,2023.0,3.0,1,130,643181.0,5,82401
2189599,2468.0,2917.0,2.0,1,130,2.0,5,73329
2189600,5552.0,6151.0,1.0,1,68,NaN,5,129271
